In [ ]:
# Standard data science libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import copy as cp
import time

plt.close('all')
sns.set(color_codes=True)

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize, least_squares

from sklearn.linear_model import Lasso, LinearRegression

import scipy.stats as st
from scipy import signal
import scipy
from scipy.interpolate import interp1d
from scipy.fftpack import fft, ifft, rfft, irfft

import statsmodels.api as sm

from collections import Counter

import itertools
from itertools import permutations

import sys

import pyunicorn
from pyunicorn import timeseries
from pyunicorn.timeseries.surrogates import Surrogates

from matplotlib import cm

import os
from os import listdir
from os.path import isfile, join

import NeuronVasomotionFunc as nvf

%matplotlib inline

Functions

In [ ]:
def createLagMat(X, tau_list):
    X = np.array(X)
    num_tau = len(tau_list)
    start_ind = np.max(tau_list)
    num_rows_X = X.shape[0]
    num_rows_lag = num_rows_X - start_ind
    
    lag_X_matrix = np.zeros((num_rows_lag, num_tau))

    for i in np.arange(num_tau):        
        temp_tau = tau_list[i]
        
        lag_X_matrix[:,i] = X[(start_ind-temp_tau):(num_rows_X-temp_tau)]
    
    return lag_X_matrix

In [ ]:
def CMA_Sync(X, N, eig_values, eig_vecs, alpha, smooth):    
    m = len(eig_values);
    #print(m)
    lambda_mat_surr = np.zeros((N, len(eig_values)))

    ts = timeseries.surrogates.Surrogates(X.T, silence_level=2)
    for i in np.arange(N):
        #surr_ts = ts.refined_AAFT_surrogates(X, n_iterations=5)
        surr_ts = ts.AAFT_surrogates(X.T)
        surr_ts = surr_ts.T
        
        if smooth[0] == 1:
            surr_ts = smooth_mat(surr_ts, smooth[1])
        
        surr_cov_mat = np.cov(surr_ts.T)
        surr_eig_values, surr_eig_vecs = np.linalg.eig(surr_cov_mat);
    
        temp_sort_ind = np.argsort(surr_eig_values)
        sorted_surr_eig_values = surr_eig_values[temp_sort_ind]
    
        lambda_mat_surr[i,:] = sorted_surr_eig_values
    
    mean_lambdas_surr = np.mean(lambda_mat_surr, axis=0)
    std_lambdas_surr = np.std(lambda_mat_surr, axis=0)

    K = st.norm.ppf(1-alpha/2)
    
    thresh = K*std_lambdas_surr
    
    sync_ind = np.zeros(m)
    for t in np.arange(m):
        if eig_values[t] > (thresh[t] + mean_lambdas_surr[t]):
            sync_ind[t] = (eig_values[t] - mean_lambdas_surr[t])/(m - mean_lambdas_surr[t])
        else:
            sync_ind[t] = 0
    
    PI_mat = np.zeros((m,m))

    for i in np.arange(m):
        PI_mat[m-i-1,:] = eig_values[i]*(np.power(eig_vecs[:,i], 2))
    
    return [sync_ind, PI_mat, mean_lambdas_surr, thresh]

In [ ]:
def createTimeWindows(data, T, w):
    
    data = np.array(data);
    
    if len(data.shape) > 1:
        num_dim = data.shape[1];
    else:
        num_dim = 1;
        data = data.reshape((len(data),1))
        
    num_obs = data.shape[0];
    
    ref_beg_ind = 0;
    
    static_init_mat = np.zeros((T,num_dim,1))
    
    T_vec = []
    
    while ref_beg_ind < num_obs-T:
        
        if ref_beg_ind == 0:
            tw_mat = static_init_mat;
        else:
            tw_mat = np.concatenate((tw_mat, static_init_mat), axis=2);
        
        ref_end_ind = ref_beg_ind + T;
        
        #print(ref_end_ind)
        
        tw_mat[:,:,-1] = data[ref_beg_ind:ref_end_ind,:];
        
        ref_beg_ind = ref_end_ind - w;
    
        T_vec.append(ref_end_ind)
    
    return [tw_mat, np.array(T_vec)]

In [ ]:
def detCorrSigTimeLags(X, L, tau_list):
    L_strd = (L - np.mean(L))/np.std(L)

    lag_X_matrix = createLagMat(X, tau_list)
    
    max_tau_shift = np.max(tau_list)
    
    L_temp = L_strd[max_tau_shift:]
    
    corr_vec = np.zeros(len(tau_list));
    
    for i in np.arange(len(tau_list)):
        
        temp_corr = np.corrcoef(L_temp, lag_X_matrix[:,i]);
        corr_vec[i] = temp_corr[0,1]
        
    return [corr_vec, lag_X_matrix]

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
def smooth_mat(neur_data_raw, ma_w):
    ta_adj = ma_w-1
    
    temp_mov_av = np.zeros(neur_data_raw[ta_adj:,:].shape)
    
    for i in np.arange(neur_data_raw.shape[1]):
        temp_mov_av[:,i] = moving_average(neur_data_raw[:,i], ma_w)
        
    return temp_mov_av

In [ ]:
def StatAnalysis(data, data_raw, neuron_ind, PO2_ch, beg_ind, end_ind, alpha, lmbda, Tw, w, ma_w, max_tau, N, lasso, tl_alpha, ffq_filter, tl):     
    
    ref_data = data
    
    if neuron_ind is None:
        X = ref_data[beg_ind:end_ind, 3:]
        X_raw = data_raw[beg_ind:end_ind, 3:]
    else:
        X = ref_data[beg_ind:end_ind, 2 + np.array(neuron_ind)]
        X_raw = data_raw[beg_ind:end_ind, 2 + np.array(neuron_ind)]
    
    T = ref_data[beg_ind:end_ind, 0]
    L = ref_data[beg_ind:end_ind, PO2_ch]

    m = X.shape[1]
    bonf_alpha = alpha/m

    X_norm = (X-np.mean(X, axis = 0))/np.std(X, axis = 0)

    L_strd = (L - np.mean(L))/np.std(L)

    if ffq_filter:
        L_strd = filterFreq(norm_L, [0, 1/60], 2, 1)
    
    tau_list = np.arange(max_tau+1)

    time_window_mat_X_orig, t_vec = createTimeWindows(X, Tw, w)
    time_window_mat_L_orig, _ = createTimeWindows(L_strd, Tw, w)

    time_window_mat_X_raw, _ = createTimeWindows(X_raw, Tw, w)

    num_time_windows = time_window_mat_X_orig.shape[2]

    sync_ind_vec = np.zeros((m, num_time_windows))
    PI_mat_vec = np.zeros((m,m, num_time_windows))
    score_vec = np.zeros(num_time_windows)
    coef_mat = np.zeros((m, num_time_windows))

    time_lag_corr_mat = np.zeros((len(tau_list), m, num_time_windows));
    sig_tl_corr_mat = np.zeros((m,num_time_windows))

    for t in np.arange(num_time_windows):
        
        data_tw_filt_X_orig = time_window_mat_X_orig[:,:,t]
        data_tw_filt_L_orig = time_window_mat_L_orig[:,:,t]

        data_tw_filt_X_raw = time_window_mat_X_raw[:,:,t]
        
        X_norm_temp_orig = (data_tw_filt_X_orig - np.mean(data_tw_filt_X_orig, axis = 0))/np.std(data_tw_filt_X_orig, axis = 0)
        L_norm_temp_orig = (data_tw_filt_L_orig - np.mean(data_tw_filt_L_orig, axis = 0))/np.std(data_tw_filt_L_orig, axis = 0)

        X_norm_temp_raw = (data_tw_filt_X_raw - np.mean(data_tw_filt_X_raw, axis = 0))/np.std(data_tw_filt_X_raw, axis = 0)

        max_lag = np.max(tau_list);

        tau_shifted_L = L_norm_temp_orig[max_lag:]
        sig_lag_X_matrix = np.zeros((Tw-max_lag, m));

        for n in np.arange(m):
            X_series = X_norm_temp_orig[:,n];
            temp_corrs, temp_lag_X_matrix = detCorrSigTimeLags(X_series, L_norm_temp_orig.ravel(), tau_list);
            
            time_lag_corr_mat[:,n,t] = temp_corrs
            
            if tl > 0:
                sig_corr_ind = tl
                sig_tl_corr_mat[n,t] = temp_corrs[tl]
                sig_lag_X_matrix[:,n] = temp_lag_X_matrix[:,tl]
            else:
                sig_corr_ind = np.argmax(temp_corrs);
                sig_tl_corr_mat[n,t] = temp_corrs[sig_corr_ind]
                sig_lag_X_matrix[:,n] = temp_lag_X_matrix[:,sig_corr_ind]

        X_p = X_norm_temp_orig.T

        nan_flag = ~np.isnan(X_p[:,0])
        ind_range = np.arange(m)
        nan_filter_ind = ind_range[nan_flag]
        
        temp_corr_vec = sig_tl_corr_mat[nan_filter_ind, t]
        neg_corr_log = temp_corr_vec < 0
        
        X_p_filter = X_p[nan_filter_ind, :]
        
        cov_mat = np.cov(X_p_filter)
    
        eig_values, eig_vecs = np.linalg.eig(cov_mat)

        sort_ind = np.argsort(eig_values)
        sorted_eig_values = eig_values[sort_ind]
        sorted_eig_vecs = eig_vecs[:,sort_ind]
        
        smooth = [1,6]
        
        X_norm_temp_raw_filtered = X_norm_temp_raw[:, nan_filter_ind]
        
        sync_ind, PI_mat, mean_lambdas_surr, thresh = CMA_Sync(X_norm_temp_raw_filtered, 
                                                               N, sorted_eig_values,  
                                                               sorted_eig_vecs, bonf_alpha, smooth)
        
        if np.sum(~nan_flag) > 0:
            
            nan_indices = ind_range[~nan_flag]
            
            for a in nan_indices:
                PI_mat = np.insert(PI_mat, a, 0, axis = 0)
                PI_mat = np.insert(PI_mat, a, 0, axis = 1)
        
        sync_ind_vec[:np.sum(nan_flag),t] = sync_ind
        PI_mat_vec[:,:,t] = PI_mat
        
        sig_lag_X_matrix_filtered = sig_lag_X_matrix[:, nan_filter_ind]
        
        const_vec = np.ones((sig_lag_X_matrix_filtered.shape[0],1))
        indep_X = np.concatenate((const_vec, sig_lag_X_matrix_filtered), axis = 1)
        
        if lasso:
            temp_clf = linear_model.Lasso(alpha=lmbda)
            temp_clf.fit(indep_X, tau_shifted_L)

            temp_score = temp_clf.score(indep_X, tau_shifted_L)
            temp_coef = temp_clf.coef_
        else:
            model = sm.OLS(tau_shifted_L, indep_X, axis=1)
            res = model.fit()

            temp_score = res.rsquared
            temp_coef = res.params

        score_vec[t] = np.power(temp_score,1/2)
        coef_mat[nan_filter_ind,t] = temp_coef[1:]
    
    corr_time_lag_vecs = np.zeros((m, num_time_windows))
    for i in np.arange(num_time_windows):
        for t in np.arange(m):
            sig_ind_vec = time_lag_corr_mat[:,t,i]
            if np.max(abs(sig_ind_vec)) < tl_alpha:
                corr_time_lag_vecs[t,i] = float('nan');
            else:
                corr_time_lag_vecs[t,i] = np.argmax(sig_ind_vec)/2
                
    adj_time_vec = (t_vec-Tw/2)/(2*60)
    
    return [score_vec, coef_mat, sync_ind_vec, PI_mat_vec, adj_time_vec, sig_tl_corr_mat, corr_time_lag_vecs]

In [ ]:
def filterFreq(series, filter_window, samp_freq, exclude):
    N = len(series)

    # sample spacing
    T = samp_freq
    dt = (1/T)*samp_freq

    x = np.linspace(0.0, N*T, N)
    yf = rfft(series)
    
    if N % 2 == 0:
        xf = np.linspace(0.0, dt, N//2)
        xf_filter_temp = ((xf > filter_window[0]) & (xf <= filter_window[1]))
        xf_filter = np.concatenate((xf_filter_temp, np.flip(xf_filter_temp)))
    else:
        xf = np.linspace(0.0, dt, N//2+1)
        xf_filter_temp = ((xf > filter_window[0]) & (xf <= filter_window[1]))
        xf_filter = np.concatenate((xf_filter_temp, np.flip(xf_filter_temp[:-1])))
    
    if exclude:
        yf[xf_filter] = 0
    else:
        yf[~xf_filter] = 0

    yf_inv = irfft(yf)
    
    return yf_inv    

In [ ]:
def true_sync(PI_mat, sync_ind, sig_tl_corr_mat, thresh):
    
    m = PI_mat.shape[0]
    T = PI_mat.shape[2]
    sample_m = PI_mat.shape[1]
    
    new_sync_ind = np.zeros((m,T))
    
    for i in np.arange(m):
        for t in np.arange(T):
            if np.sum(PI_mat[i,:,t] > thresh) == sample_m:
                if np.sum(sig_tl_corr_mat[:,t] > thresh) == sample_m:
                    new_sync_ind[i,t] = sync_ind[-(i+1),t]
    
    return new_sync_ind

In [ ]:
def norm(X):
    norm_X = (X - np.mean(X, axis=0))/np.std(X, axis=0)
    return norm_X

Scripts

In [ ]:
### Individual Experiment Analysis ###

## Set parameters for data analysis

PO2_ch = 1       # PO2 channel (always set to 1, since there is only one channel)
beg_ind = 0      # Beginning time index of data (in units of .5 seconds)
end_ind = 1200   # Ending time index of data (in units of .5 seconds)
alpha = .05      # p-value level for Bonferonni correction
lmbda = .1       # Regularization parameter for lasso regression
Tw = 240         # Size of time window for rolling window analysis
w = Tw - 1       # Overlap of time windows
ma_w = 6         # Moving average window
max_tau = 10     # Maximum time lag for determining correlations
N = 100          # Number of surrogates to generate for Synhronization index calculations
lasso = 1        # Indicator variable to indicate whether to run Lasso regression. 1 = Lasso and 0 = OLS
tl_alpha = 0     # Minimum threshold correlation threshold to record the time lag. Always set to zero.
ffq_filter = 0   # Indicator on whether to apply smoothing in the freqyency domain (for this paper, always zero) 
tl = -1          # Amount of time lag set for ALL neurons. tl = -1 applies the usual case-by-case peak picking algorithm for choosing time lags.

# Set neurons to be analysed as a list object.
# None = All neurons in data
neuron_inds = None 

synch = 0        # Parameter solely for naming figure outputs. Synch = 1 if selected neuron inds are synchronized

## Insert path for SU data
smooth_path = '[Folder path here]'

## Insert SU file name
temp_file = '[File name here]'

temp_smooth_path = join(smooth_path, temp_file)
temp_data_smooth = np.genfromtxt(temp_smooth_path, delimiter = ',')

## Raw data is only used to generate surrogates for the synchronization index calculations.
## For the purposes of this paper, we set raw data equal to smoothed data.
temp_raw_path = join(smooth_path, temp_file)
temp_data_raw = np.genfromtxt(temp_raw_path, delimiter = ',')

time_vec = temp_data_smooth[:,0]
L = temp_data_smooth[:, PO2_ch]
norm_L = (L - np.mean(L))/np.std(L)
temp_norm_L = nvf.filterFreq(norm_L, [0,2/60], 2, 1)

score_vec, coef_mat, sync_ind_vec, PI_mat_vec, adj_time_vec, sig_tl_corr_mat, corr_time_lag_vecs \
        = StatAnalysis(temp_data_smooth, temp_data_raw, neuron_inds, PO2_ch, beg_ind, end_ind, alpha, 
                       lmbda, Tw, w, ma_w, max_tau, N, lasso, tl_alpha, ffq_filter, tl)

In [ ]:
m = coef_mat.shape[0]

sig_thresh = .5
cluster_ind = 0

test_sync = true_sync(PI_mat_vec, sync_ind_vec, sig_tl_corr_mat, -1)

output_data = 1

exp_name = temp_file[:-20]

viridis = cm.get_cmap('viridis', 256)
seismic = cm.get_cmap('seismic', 256)

fig, axes = plt.subplots(6, 1, figsize=(15, 15))

axes[0].plot(adj_time_vec, score_vec)
axes[1].plot(adj_time_vec, test_sync[cluster_ind,:])
axes[2].pcolormesh(PI_mat_vec[cluster_ind,:,:], cmap = viridis, rasterized=True, vmin=0, vmax=1)
axes[3].pcolormesh(sig_tl_corr_mat, cmap = seismic, rasterized=True, vmin=-1, vmax=1)
axes[4].pcolormesh(coef_mat, cmap = seismic, rasterized=True, vmin=-1, vmax=1)

ma_w_tl = 9
ta_adj = ma_w_tl - 1

smooth_tl_ind = smooth_mat(corr_time_lag_vecs.T, ma_w_tl)
smooth_tl_ind = smooth_tl_ind.T

shifted_adj_time_vec = adj_time_vec[ta_adj:]

for i in np.arange(m):
    axes[5].plot(shifted_adj_time_vec, smooth_tl_ind[i,:], \
                 label = "Neuron " + str(i+1))

fig.tight_layout()

if output_data:
    
    if neuron_inds == None:
        out_cases_folder = '/home/evan/Projects/NeuronVasomotion/Synchronization_Analysis/' + exp_name + '_fullpop'
    elif synch == 0:
        out_cases_folder = '/home/evan/Projects/NeuronVasomotion/Synchronization_Analysis/' + exp_name + '_subpop_nonsynch'
    else:
        out_cases_folder = '/home/evan/Projects/NeuronVasomotion/Synchronization_Analysis/' + exp_name + '_subpop'
    
    if not(os.path.exists(out_cases_folder)):
        os.makedirs(out_cases_folder)
    
    if neuron_inds == None:
        fig.savefig(join(out_cases_folder, exp_name + "_fig_fullpop.png"));
    elif synch == 0:
        fig.savefig(join(out_cases_folder, exp_name + "_fig_subpop_nonsynch.png"));
    else:
        fig.savefig(join(out_cases_folder, exp_name + "_fig_subpop.png"));
    
    mat_output = np.c_[adj_time_vec, score_vec, test_sync[0,:].reshape(-1,1)]
    
    if neuron_inds == None:
        neuron_inds_temp = np.arange(1, temp_data_smooth.shape[1]-2)
    else:
        neuron_inds_temp = neuron_inds
        
    neuron_data = temp_data_smooth[:,2+np.array(neuron_inds_temp)]
    
    data_output = np.c_[time_vec, norm_L, neuron_data]
    
    h1 = "Time (min), Score (r), Synchronization Index"
    np.savetxt(join(out_cases_folder, exp_name + "_fig_data.csv"), mat_output, delimiter = ',', header = h1, comments='')
    
    # Create second header
    for i in np.arange(neuron_data.shape[1]):
        if i == 0:
            h2 = "SU" + str(neuron_inds_temp[i])
        else:
            h2 += ", SU" + str(neuron_inds_temp[i])
    
    h3 = "Time (s), PO2, " + h2
    
    np.savetxt(join(out_cases_folder, exp_name + "_timelag.csv"), smooth_tl_ind.T, delimiter = ',', header = h2, comments='')
    np.savetxt(join(out_cases_folder, exp_name + "_neuron_series.csv"), data_output, delimiter = ',', header = h3, comments='')
    np.savetxt(join(out_cases_folder, exp_name + "_corr_mat.csv"), sig_tl_corr_mat.T, delimiter = ',', header = h2, comments='')
    np.savetxt(join(out_cases_folder, exp_name + "_coef_mat.csv"), coef_mat.T, delimiter = ',', header = h2, comments='')

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(15, 5))

if neuron_inds == None:
    neuron_inds_temp = np.arange(1, temp_data_smooth.shape[1]-2)
else:
    neuron_inds_temp = neuron_inds

neuron_data = temp_data_smooth[:,2+np.array(neuron_inds_temp)]

num_neurons = neuron_data.shape[1]
max_ind = np.argmax(score_vec)
tl_vec = corr_time_lag_vecs[:, max_ind]

max_tau = 10
Tw = 250

beg_ind = max_ind
end_ind = max_ind + Tw

neur_num = 0
tl = int(tl_vec[neur_num])

mod_time_vec = time_vec[beg_ind+max_tau:end_ind]

window_L = norm_L[beg_ind:end_ind]
window_L = norm(window_L)

tw_norm_L = window_L[max_tau:]

normed_neuron_data = norm(neuron_data[beg_ind:end_ind])

tl_neuron_data = np.zeros((end_ind-beg_ind-max_tau, num_neurons))

for i in np.arange(num_neurons):
    temp_tl = int(tl_vec[i]*2)
    tl_neuron_data[:, i] = normed_neuron_data[max_tau-temp_tl:Tw-temp_tl,i]

axes.plot(mod_time_vec, tw_norm_L)

axes.plot(mod_time_vec, tl_neuron_data[:,neur_num])

const_vec = np.ones((tl_neuron_data.shape[0],1))
indep_X = np.concatenate((const_vec, tl_neuron_data), axis = 1)

temp_clf = linear_model.Lasso(alpha=.1)
temp_clf.fit(indep_X , tw_norm_L)
temp_score = temp_clf.score(indep_X, tw_norm_L)
temp_coef = temp_clf.coef_

print(temp_coef)
print(np.power(temp_score, 1/2))

In [ ]:
OLS_series = temp_clf.predict(indep_X)
normed_OLS_series = norm(OLS_series)

plt.plot(tw_norm_L)
plt.plot(normed_OLS_series)

In [ ]:
output_mat = np.concatenate((mod_time_vec.reshape(-1,1),tw_norm_L.reshape(-1,1),
                             tl_neuron_data, normed_OLS_series.reshape(-1,1), ), axis=1)
    
if not(os.path.exists(out_cases_folder)):
    os.makedirs(out_cases_folder)

output_path = join(out_cases_folder, exp_name + "_MaxTWData.csv")

import csv

with open(output_path,'w') as result_file:
    wr = csv.writer(result_file, dialect='excel')

    header = ["Time", "PO2"]
    
    for h in np.arange(num_neurons):
        temp_SU = "SU" + str(neuron_inds_temp[h])
        header.append(temp_SU)
        
    header.append("Regression Output")
    
    wr.writerow(header)

    for i in np.arange(output_mat.shape[0]):
        wr.writerow(output_mat[i,:])